## References

see README.md

In [1]:
import os
import itertools


import datetime

import editdistance
import numpy as np

import pylab
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = 'image_ocr'



np.random.seed(55)

In [3]:
# Text_Image is the original generator. It creates images programaticaly. 
# Script_Image takes handwritten words from the IAM database.
generator_choice =  "Script_Image" # "Text_Image" #
import generator_text_image as GTI
import generator_iam_words as IAM

code under development ...
click debug in visual studio code


In [4]:
import ctc_drop_first_2
import cnn_rnn_model

In [5]:



# For a real OCR application, this should be beam search with a dictionary
# and language model.  For this example, best path is sufficient.

def decode_batch(test_func, word_batch):
    out = test_func([word_batch])[0]
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        if generator_choice ==  "Script_Image":
            outstr = IAM.labels_to_text(out_best)
        else:
            outstr = GTI.labels_to_text(out_best)
        ret.append(outstr)
    return ret


In [6]:
class VizCallback(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, num_display_words=6):
        self.test_func = test_func
        self.output_dir = os.path.join(
            OUTPUT_DIR, run_name)
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def show_edit_distance(self, num):
        num_left = num
        mean_norm_ed = 0.0
        mean_ed = 0.0
        while num_left > 0:
            word_batch = next(self.text_img_gen)[0]
            num_proc = min(word_batch['the_input'].shape[0], num_left)
            decoded_res = decode_batch(self.test_func, word_batch['the_input'][0:num_proc])
            for j in range(num_proc):
                edit_dist = editdistance.eval(decoded_res[j], word_batch['source_str'][j])
                mean_ed += float(edit_dist)
                mean_norm_ed += float(edit_dist) / len(word_batch['source_str'][j])
            num_left -= num_proc
        mean_norm_ed = mean_norm_ed / num
        mean_ed = mean_ed / num
        print('\nOut of %d samples:  Mean edit distance: %.3f Mean normalized edit distance: %0.3f'
              % (num, mean_ed, mean_norm_ed))

    def on_epoch_end(self, epoch, logs={}):
        self.model.save_weights(os.path.join(self.output_dir, 'weights%02d.h5' % (epoch)))
        self.show_edit_distance(256)
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, word_batch['the_input'][0:self.num_display_words])
        if word_batch['the_input'][0].shape[0] < 256:
            cols = 2
        else:
            cols = 1
        for i in range(self.num_display_words):
            plt.subplot(self.num_display_words // cols, cols, i + 1)
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['the_input'][i, 0, :, :]
            else:
                the_input = word_batch['the_input'][i, :, :, 0]
            plt.imshow(the_input.T, cmap='Greys_r')
            plt.xlabel('Truth = \'%s\'\nDecoded = \'%s\'' % (word_batch['source_str'][i], res[i]))
        fig = pylab.gcf()
        fig.set_size_inches(10, 13)
        plt.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        plt.close()


In [7]:
img_gen = None
def train(run_name, start_epoch, stop_epoch, img_w):
    global img_gen
    img_h = 64
    
    # Input Parameters

    words_per_epoch = 16000
    val_split = 0.2
    
    # 16000 * 0.2 = 3200
    val_words = int(words_per_epoch * (val_split))

    # Network parameters
    minibatch_size = 32
    pool_size = 2
    
    # (16000 - 3200) // 32 = 400
    steps_per_epoch = (words_per_epoch - val_words) // minibatch_size


    if generator_choice == "Text_Image":
        fdir = os.path.dirname(get_file('wordlists.tgz',
                                        origin='http://www.mythic-ai.com/datasets/wordlists.tgz', untar=True))

        img_gen = GTI.TextImageGenerator(monogram_file=os.path.join(fdir, 'wordlist_mono_clean.txt'),
                                     bigram_file=os.path.join(fdir, 'wordlist_bi_clean.txt'),
                                     minibatch_size=minibatch_size,
                                     img_w=img_w,
                                     img_h=img_h,
                                     downsample_factor=(pool_size ** 2),
                                     val_split=words_per_epoch - val_words
                                     )
    elif generator_choice == "Script_Image":
        img_gen = IAM.IAM_Word_Generator(minibatch_size = 32, img_w = 128, img_h = 64, downsample_factor=4, absolute_max_string_len=16)
    else:
        assert False

    model, input_data, y_pred = cnn_rnn_model.make_model(img_w, img_h, pool_size, img_gen.get_output_size(), img_gen.absolute_max_string_len)
    
    
    model.summary() # print summary of model

    # clipnorm seems to speeds up convergence
    sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
    
    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
    if start_epoch > 0:
        weight_file = os.path.join(OUTPUT_DIR, os.path.join(run_name, 'weights%02d.h5' % (start_epoch - 1)))
        model.load_weights(weight_file)
    # captures output of softmax so we can decode the output during visualization
    test_func = K.function([input_data], [y_pred])

    viz_cb = VizCallback(run_name, test_func, img_gen.next_val())

    model.fit_generator(generator=img_gen.next_train(),
                        steps_per_epoch=steps_per_epoch,
                        epochs=stop_epoch,
                        validation_data=img_gen.next_val(),
                        validation_steps=val_words // minibatch_size,
                        callbacks=[viz_cb, img_gen],
                        initial_epoch=start_epoch)


### Start the training

1080ti run times: 
- 12 minutes for 20/20 epoch
- 25 minutes for 25/25 epoch


In [8]:
run_name = datetime.datetime.now().strftime('%Y:%m:%d:%H:%M:%S')
train(run_name, 0, 200, 128)
# increase to wider images and start at epoch 20. The learned weights are reloaded
img_gen.set_img_w(512)
train(run_name, 20, 25, 512)

Instructions for updating:
Colocations handled automatically by placer.
Tensor("the_labels:0", shape=(?, 16), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
______________________________________________________________________________________

400/400 [==============================] - 32s 79ms/step - loss: 4.4421 - val_loss: 4.5507

Out of 256 samples:  Mean edit distance: 1.176 Mean normalized edit distance: 0.319
Epoch 13/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 4.2834 - val_loss: 4.3134

Out of 256 samples:  Mean edit distance: 1.262 Mean normalized edit distance: 0.349
Epoch 14/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 4.2023 - val_loss: 4.0855

Out of 256 samples:  Mean edit distance: 1.242 Mean normalized edit distance: 0.300
Epoch 15/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 4.0699 - val_loss: 3.8709

Out of 256 samples:  Mean edit distance: 0.961 Mean normalized edit distance: 0.284
Epoch 16/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 3.8251 - val_loss: 3.6475

Out of 256 samples:  Mean edit distance: 1.012 Mean normalized edit distance: 0.252
Epoch 17

400/400 [==============================] - 32s 79ms/step - loss: 1.6130 - val_loss: 1.4386

Out of 256 samples:  Mean edit distance: 0.320 Mean normalized edit distance: 0.089
Epoch 53/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.5730 - val_loss: 1.5184

Out of 256 samples:  Mean edit distance: 0.426 Mean normalized edit distance: 0.095
Epoch 54/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.5864 - val_loss: 1.4752

Out of 256 samples:  Mean edit distance: 0.465 Mean normalized edit distance: 0.127
Epoch 55/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.5955 - val_loss: 1.5887

Out of 256 samples:  Mean edit distance: 0.449 Mean normalized edit distance: 0.105
Epoch 56/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.5564 - val_loss: 1.4568

Out of 256 samples:  Mean edit distance: 0.484 Mean normalized edit distance: 0.129
Epoch 57

400/400 [==============================] - 32s 79ms/step - loss: 1.2574 - val_loss: 1.3209

Out of 256 samples:  Mean edit distance: 0.406 Mean normalized edit distance: 0.123
Epoch 93/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.2031 - val_loss: 1.0529

Out of 256 samples:  Mean edit distance: 0.395 Mean normalized edit distance: 0.104
Epoch 94/200
on_epoch_begin
400/400 [==============================] - 31s 79ms/step - loss: 1.1856 - val_loss: 0.9743

Out of 256 samples:  Mean edit distance: 0.258 Mean normalized edit distance: 0.071
Epoch 95/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.2532 - val_loss: 1.1716

Out of 256 samples:  Mean edit distance: 0.336 Mean normalized edit distance: 0.081
Epoch 96/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.1458 - val_loss: 1.0144

Out of 256 samples:  Mean edit distance: 0.297 Mean normalized edit distance: 0.077
Epoch 97

400/400 [==============================] - 32s 80ms/step - loss: 1.2514 - val_loss: 1.2792

Out of 256 samples:  Mean edit distance: 0.348 Mean normalized edit distance: 0.089
Epoch 133/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.1689 - val_loss: 1.1048

Out of 256 samples:  Mean edit distance: 0.430 Mean normalized edit distance: 0.103
Epoch 134/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.1812 - val_loss: 1.1309

Out of 256 samples:  Mean edit distance: 0.355 Mean normalized edit distance: 0.103
Epoch 135/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.2753 - val_loss: 1.2610

Out of 256 samples:  Mean edit distance: 0.355 Mean normalized edit distance: 0.085
Epoch 136/200
on_epoch_begin
400/400 [==============================] - 32s 80ms/step - loss: 1.2781 - val_loss: 1.2460

Out of 256 samples:  Mean edit distance: 0.324 Mean normalized edit distance: 0.085
Epoc

400/400 [==============================] - 32s 79ms/step - loss: 1.1916 - val_loss: 1.1030

Out of 256 samples:  Mean edit distance: 0.352 Mean normalized edit distance: 0.096
Epoch 173/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.5161 - val_loss: 1.2014

Out of 256 samples:  Mean edit distance: 0.312 Mean normalized edit distance: 0.087
Epoch 174/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.3063 - val_loss: 1.4894

Out of 256 samples:  Mean edit distance: 0.422 Mean normalized edit distance: 0.109
Epoch 175/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.3005 - val_loss: 1.2645

Out of 256 samples:  Mean edit distance: 0.465 Mean normalized edit distance: 0.120
Epoch 176/200
on_epoch_begin
400/400 [==============================] - 32s 79ms/step - loss: 1.2419 - val_loss: 4.4700

Out of 256 samples:  Mean edit distance: 1.039 Mean normalized edit distance: 0.271
Epoc

on_train_begin
Epoch 21/25
on_epoch_begin


ValueError: Error when checking input: expected the_input to have shape (512, 64, 1) but got array with shape (128, 64, 1)